In [ ]:
%pip install requests pandas matplotlib seaborn bs4 lxml html5lib geopy scikit-learn

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://fr.wikipedia.org/wiki/Liste_des_gares_desservies_par_TGV"

html = requests.get(url).content
soup = BeautifulSoup(html, "html.parser")

tables = soup.find_all("table", {"class": "wikitable"})

all_rows = []

for table in tables:
    headers = [th.text.strip() for th in table.find_all("th")]
    
    for row in table.find_all("tr")[1:]:
        cells = [td.text.strip() for td in row.find_all(["td", "th"])]
        if len(cells) == len(headers):
            all_rows.append(dict(zip(headers, cells)))

df = pd.DataFrame(all_rows)

print(df.head())


In [ ]:
import pandas as pd
import requests
import io

url = "https://fr.wikipedia.org/wiki/Liste_des_gares_desservies_par_TGV"

# 1. On se fait passer pour un navigateur web pour ne pas être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

try:
    # On fait la requête avec les headers et sans vérification SSL
    response = requests.get(url, headers=headers, verify=False)
    
    # On vérifie si la page a bien été chargée (Code 200 = OK)
    if response.status_code == 200:
        print("Page récupérée avec succès !")
        
        # 2. On utilise io.StringIO pour que Pandas lise le texte comme un fichier
        # 3. 'match="Département"' force Pandas à ne garder que les tables qui contiennent ce mot
        dfs = pd.read_html(io.StringIO(response.text), match="Département")
        
        if len(dfs) > 0:
            df_gares = dfs[0] # On prend la première table trouvée
            print("-" * 30)
            print(f"Tableau trouvé avec {len(df_gares)} gares !")
            print(df_gares.head())
        else:
            print("Aucune table correspondante trouvée.")
            
    else:
        print(f"Erreur : Wikipédia a renvoyé le code {response.status_code}")

except Exception as e:
    print(f"Une erreur s'est produite : {e}")

In [5]:
import requests
import time

def geocode_osm(query):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": query,
        "format": "json",
        "limit": 1
    }
    headers = {
        "User-Agent": "projet-academique"
    }

    r = requests.get(url, params=params, headers=headers)
    if r.status_code == 200 and r.json():
        res = r.json()[0]
        return float(res["lat"]), float(res["lon"])
    return None, None


In [6]:
lats, lons = [], []

for _, row in df.iterrows():
    query = f"{row['Gare']}, {row['Département']}, France"
    lat, lon = geocode_osm(query)
    lats.append(lat)
    lons.append(lon)
    time.sleep(1)  # obligatoire


In [8]:
df["latitude"] = lats
df["longitude"] = lons

df.head(5)

,latitude,longitude


In [9]:
print(df.columns)

Index(['latitude', 'longitude'], dtype='object')
